# <font color='blue'> The script objective:</font>
#### Basically it get all your files in your drive in one place, the script access your google drive account, then start to collect all files in your drive and build dataframe with all your files and finally cleans the data, 

In [ ]:
# Importing the needed packages:
import numpy as np
import pandas as pd
import requests
import os
import pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Accessing the GoogleDrive API:
gauth = GoogleAuth()

# client_secrets.json need to be in the same directory as the script
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

In [2]:
# Creating the datafrmae of the mime_types table:
mime_types = pd.DataFrame()
files = {}
# Looping over the files on the web to collect it all:
for i in ['application', 'audio', 'font', 'image', 'message' ,'model' ,'multipart' ,'text' ,'video']:
    p = 'https://www.iana.org/assignments/media-types/'
    # Mapping the mime_type type to their link to download it:
    files[i] = str(p + i + '.csv')
    for k,v in files.items():
        # Loading, wrangling, appending the frames togther in one dataframe:
        try:
            globals()[k] = pd.DataFrame(pd.read_csv(v))
            globals()[k]['type'] = i
            mime_types = mime_types.append(globals()[k])
            # deleting the dataframes as we have one dataframe for all.
            del globals()[k]
        except:
            print('({}) file does not exsist on the web check the source: {}'.format(i,str(p + i)))
            pass

In [84]:
# Create the recurrsive function to expand and combine the folers-tree:
def folders(parent):
    files = drive.ListFile({'q': "'%s' in parents and trashed=false" % parent}).GetList()
    for file in files:
        if file['mimeType'] != 'application/vnd.google-apps.folder':
            # Defining my_drive to be global value to be used here:
            global my_drive
            # Assining the folder_id to map it later with folder details:
            file['folder_id'] = parent
            my_drive = my_drive.append(file, ignore_index=True)
        else:
            # If folder then re-call the function again to open the sub-tree:
            folders(file['id'])
            pass

# Create the dataframe which will contain the whole files of your drive:
my_drive = pd.DataFrame()
# Firstly will launch the root drive to start the tree-structure of the drive: 
for root_file in drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList():
    # If the objects are files then will add it to the dataframe:
    if root_file['mimeType'] != 'application/vnd.google-apps.folder':
        my_drive = my_drive.append(root_file, ignore_index=True)
        my_drive['folder_id'] = root_file['id']
        # If the objects are folders then call the folders tree function:
    else:
        folders(root_file['id'])

In [48]:
# Joining the mimetypes dataframe to my_drive then re-joining with google-api mime-types:
my_drive = pd.merge(
    left = pd.merge(
        left = my_drive, right = mime_types, how = 'left', 
        left_on='mimeType', right_on='Template'
        ).drop(['Reference','Template'], axis=1).fillna('')
    ,right = pd.DataFrame(pd.read_html(
        requests.get('https://developers.google.com/drive/api/v3/mime-types').content)[-1]).fillna(''),
    how='left', left_on='mimeType', right_on='MIME Type'
).drop(['MIME Type'], axis=1).fillna('')

In [ ]:
# Wrangling the Name & Type column and remove the Description column:
my_drive['Name'] = np.where(my_drive['Name'] == '', my_drive['Description'], 
                            np.where(my_drive['Name'] == '', 'image/jpeg', my_drive['Name']))
my_drive['type'] = np.where(my_drive['type'] == '', 'drive', my_drive['type'])
my_drive = my_drive.drop(['Description'], axis=1)